In [1]:
import argparse
import cv2
import os
import numpy as np
from tqdm import tqdm
import re
import imutils
from utils import *

In [2]:
def write_next_img(filename, txtname):
    f = open(txtname, "a")
    f.write("\n"+filename+" ")
    f.close()
    
def write_rect(rect):
    global TXT
    f = open(TXT, "a")
    f.write(np.array_str(rect)+" ")
    f.close()

def click_and_crop(event, x, y, flags, param):
    global rect, tip_x, tip_y, clone
    h,w,_ = clone.shape
    if event == cv2.EVENT_LBUTTONDOWN:
        rect=np.concatenate((rect,[x,y]))
        if len(rect)==4:
            write_rect(rect)
            rect=np.array([])
    elif event == cv2.EVENT_MOUSEMOVE:
        white=(255,255,255)
        clone  = image.copy()
        if len(rect)==2:
            cv2.rectangle(clone, tuple(map(int,rect)), (x,y), (0,255,0))
        cv2.line(clone, (0,y),(w,y),white)
        cv2.line(clone, (x,0),(x,h),white)
        cv2.imshow("image", clone)

# Set scene, get paths and read resolution

In [3]:
SCENE = "dark_1"

DIR = get_scene_dir_path(SCENE)
TXT = get_scene_labels_path(SCENE)

_,_,_,RESOLUTION,LABEL_RESOLUTION=read_info(DIR)

# Label

In [5]:
filenames = sorted(os.listdir(DIR))
filenames=[f for f in filenames if f != "info.txt"]
#filenames=["r_2_99.jpg"]
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop)

#variables
rect = np.array([])
tip_x, tip_y = (0,0)

for i in tqdm(range(len(filenames))):
    filename=filenames[i]
    print(filename)
    path=DIR+filename
    image = cv2.imread(path)
    #image = imutils.rotate(image, 1.5)
    if RESOLUTION!=LABEL_RESOLUTION:
        image = cv2.resize(image, LABEL_RESOLUTION)
    clone = image.copy()
    write_next_img(filename, TXT)
    cv2.imshow("image", clone)
    cv2.waitKey(0)
    rect = np.array([])
cv2.destroyAllWindows()

  0%|          | 0/5 [00:00<?, ?it/s]

IMG_20200228_103440.jpg


 40%|████      | 2/5 [00:02<00:04,  1.62s/it]

IMG_20200228_103442.jpg
IMG_20200228_103443.jpg


 60%|██████    | 3/5 [00:02<00:02,  1.17s/it]

IMG_20200228_103445.jpg


 80%|████████  | 4/5 [00:02<00:00,  1.11it/s]

IMG_20200228_103446.jpg


100%|██████████| 5/5 [00:03<00:00,  1.39it/s]


# Display labelled

In [4]:
labels = read_labels(SCENE)
cv2.namedWindow("image")
for label in labels:
    filename,point_list=label
    path=DIR+filename
    image = cv2.imread(path)
    if RESOLUTION!=LABEL_RESOLUTION:
        image = cv2.resize(image, LABEL_RESOLUTION) 
    for points in point_list:
        p1,p2=points
        cv2.rectangle(image, p1, p2, (0,255,0))
    cv2.imshow("image", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()